# README (COMMON) — Camera calibration + live measurement (Chessboard, phone stream or webcam)

## What this notebook does
This notebook is split into 4 code “blocks” (cells) that together let you:
1) Open a phone MJPEG stream (or webcam fallback)
2) Optionally capture calibration frames
3) Calibrate (or load) camera intrinsics (K, dist)
4) Optionally sanity-check undistortion (side-by-side mosaic)
5) Run a live measurement tool that maps 4 clicked image points onto the chessboard plane (meters) and reports width/height (mm)

## Requirements
- Python 3.x
- Packages:
  - `opencv-python`
  - `numpy`

## Files / outputs
- `calib_frames_phone/` (folder): saved calibration images if you capture frames
- `camera_intrinsics_phone.npz` (file): saved intrinsics (K, dist, image_size, metadata)

## Chessboard assumptions
- Board is 8×8 squares → **7×7 inner corners**: `PATTERN_SIZE = (7, 7)`
- Square size is derived from `BOARD_OUTER_MM / 8` → `SQUARE_MM`
- Units:
  - Calibration object points are in **meters**
  - Live measurement prints **mm**

## How to run (typical workflows)

### Workflow A (already calibrated)
1) **Block 1**: leave defaults; ensure `INTRINSICS_PATH` exists.
2) **Block 2**: keep `PRE_CALIBRATED=True` (loads intrinsics)
3) (Optional) **Block 3**: set `SHOW_COMPARISON=True` to sanity-check undistortion
4) **Block 4**: run live measurement and click 4 points to measure

### Workflow B (calibrate from scratch)
1) **Block 1**: set `PRE_CALIBRATED=False` and verify:
   - `CAM_BASE` points to your phone-stream control page
   - `PATTERN_SIZE`, `BOARD_OUTER_MM` match your printed board
2) **Block 2**(A): capture ~15–30 frames with varied angles/distances (SPACE to save)
3) **Block 2**(B): calibrate; saves `camera_intrinsics_phone.npz`
4) (Optional) **Block 3**: sanity-check undistortion
5) **Block 4**: run live measurement

## Controls (reused across relevant blocks)
- ESC: quit
- Block 2 capture: SPACE save frame
- Block 4 measurement:
  - Left-click: add / drag points
  - Right-click: toggle zoom window
  - C: clear points
  - P: print last measurement to console

## Troubleshooting quick hits
- **Phone page opens in browser, but OpenCV can’t read stream**:
  - Open `CAM_BASE` in browser → right-click video → copy “video address”
  - Set `CAM_BASE` to that direct stream URL or extend `COMMON_STREAM_PATHS`
- **No chessboard detections**:
  - Improve lighting, reduce glare, fill more of frame, add angles
  - Enable `CLAHE_ON=True` (already in your config)
- **Live measurement says board NOT FOUND**:
  - Move board to be more fronto-parallel, improve contrast
  - Wait for periodic re-detection (`DETECT_EVERY_N`)


# Block 1 — Shared config + helpers (used by all cells)

## Purpose
Defines all shared configuration and helper functions used by later blocks:
- Chessboard geometry (square size, inner-corner pattern)
- Phone stream discovery helpers that scrape `CAM_BASE` and try common MJPEG endpoints
- Object-point generator for calibration (`mk_object_points`)

## Key settings to change
- `PRE_CALIBRATED`:
  - `True`: skip capture + calibration; load `camera_intrinsics_phone.npz`
  - `False`: capture and calibrate from images in `calib_frames_phone/`
- `SHOW_COMPARISON`:
  - `True`: enable Block 3 undistortion mosaic
- `CAM_BASE`: base page for your phone stream (IP Webcam / DroidCam-like servers)
- `BOARD_OUTER_MM`, `PATTERN_SIZE`:
  - Must match your printed chessboard
  - 8×8 squares → `PATTERN_SIZE=(7,7)`
- Output paths:
  - `CALIB_DIR` folder for captured frames
  - `INTRINSICS_PATH` for saved intrinsics `.npz`

## What it provides to downstream blocks
- `open_phone_cap()` → returns `(cap, chosen_url, first_frame)`
- `mk_object_points(pattern_size, square_m)` → 3D chessboard corner points in meters

## Notes / assumptions
- Stream endpoint discovery is best-effort:
  - It scrapes HTML for `href/src` URLs containing common MJPEG keywords
  - It tries `COMMON_STREAM_PATHS` fallbacks
- If none work, it raises an error telling you how to copy the direct stream URL from the browser.


In [ ]:
# ============================
# TOP CELL — Shared config + helpers (used by all cells below)
# ============================

import cv2
import numpy as np
import glob
import re
import urllib.request
import urllib.parse
from pathlib import Path

# ----------------------------
# Notebook toggles
# ----------------------------
PRE_CALIBRATED = False
SHOW_COMPARISON = True

# ----------------------------
# Board config
# ----------------------------
BOARD_OUTER_MM = 336.0
SQUARE_MM = BOARD_OUTER_MM / 8.0
SQUARE_M = SQUARE_MM / 1000.0
PATTERN_SIZE = (7, 7)  # 8x8 squares => 7x7 inner corners

# ----------------------------
# Phone stream base page
# ----------------------------
CAM_BASE = "http://192.168.1.70:8080/"  # base control page that works in your browser

# ----------------------------
# Files
# ----------------------------
CALIB_DIR = Path("calib_frames_phone")
INTRINSICS_PATH = "camera_intrinsics_phone.npz"

# ----------------------------
# Capture preview sizing
# ----------------------------
PREVIEW_MAX_W = 1600
PREVIEW_MAX_H = 900

# ----------------------------
# Stream endpoint discovery
# ----------------------------
COMMON_STREAM_PATHS = [
    "video",
    "videofeed",
    "mjpeg", "mjpegfeed",
    "live", "stream",
    "?action=stream",
    "video?x.mjpeg",
    "mjpg/video.mjpg",
]

# ----------------------------
# Live measurement config (CELL 3 reads these)
# ----------------------------
CAM_INDEX = 0
CAPTURE_W = 3840
CAPTURE_H = 2160

# Detection cadence / preprocess
DETECT_EVERY_N = 12
DETECT_SCALE = 0.45
CLAHE_ON = True
SB_FIRST = True

# Tracking / smoothing
TRACK_MIN_FRAC = 0.75
CORNER_EMA_ALPHA = 0.20
REDETECT_ON_TRACK_FAIL = True

# LK safety rails (prevents collapse / shoot-off)
FB_ERR_MAX_PX = 1.25
MAX_JUMP_PX = 30.0

# Display sizing
DISPLAY_MAX_W = 1600
DISPLAY_MAX_H = 900

# Point rendering
POINT_RING_R = 8
POINT_RING_TH = 2
POINT_CENTER_R = 1
POINT_HIT_R_FULL = 12
LABEL_SCALE = 0.6

# Zoom
zoom_factor = 4
zoom_radius = 180
zoom_win = "zoom"

# ----------------------------
# URL discovery helpers
# ----------------------------
def _fetch_html(url, timeout=3.0):
    req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req, timeout=timeout) as r:
        return r.read().decode("utf-8", errors="ignore")

def _extract_stream_urls_from_html(base_url, html):
    candidates = set()
    for m in re.finditer(r'(?:href|src)\s*=\s*["\']([^"\']+)["\']', html, flags=re.IGNORECASE):
        u = m.group(1).strip()
        if not u:
            continue
        low = u.lower()
        if any(k in low for k in ["mjpeg", "mjpg", "videofeed", "video", "stream", "action=stream"]):
            candidates.add(urllib.parse.urljoin(base_url, u))
    return list(candidates)

def _try_open_url(url):
    backends = [None, cv2.CAP_MSMF]
    try:
        backends.append(cv2.CAP_FFMPEG)
    except Exception:
        pass

    for be in backends:
        try:
            cap = cv2.VideoCapture(url) if be is None else cv2.VideoCapture(url, be)
        except Exception:
            continue
        if cap is None or not cap.isOpened():
            try:
                cap.release()
            except:
                pass
            continue
        ok, frame = cap.read()
        if ok and frame is not None and frame.size > 0:
            return cap, frame
        cap.release()
    return None, None

def open_phone_cap():
    base = CAM_BASE if CAM_BASE.endswith("/") else (CAM_BASE + "/")
    html = _fetch_html(base, timeout=3.0)
    scraped = _extract_stream_urls_from_html(base, html)
    fallbacks = [urllib.parse.urljoin(base, p) for p in COMMON_STREAM_PATHS]

    seen = set()
    candidates = []
    for u in scraped + fallbacks:
        if u not in seen:
            seen.add(u)
            candidates.append(u)

    for u in candidates:
        cap, frame = _try_open_url(u)
        if cap is not None:
            return cap, u, frame

    raise RuntimeError(
        "Browser opens CAM_BASE, but OpenCV can't find a readable stream endpoint.\n"
        "Fix:\n"
        "  - Open CAM_BASE in your browser\n"
        "  - Right-click the video -> Copy video address\n"
        "  - Set CAM_BASE to that direct stream URL (or paste it here)\n"
    )

def mk_object_points(pattern_size, square_m):
    nx, ny = pattern_size
    objp = np.zeros((nx * ny, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nx, 0:ny].T.reshape(-1, 2) * square_m
    return objp


# Block 2 — Capture frames + Calibrate (or load) intrinsics

## Purpose
Two-part block:
(A) Capture calibration frames from the phone stream into `CALIB_DIR` (only if `PRE_CALIBRATED=False`)
(B) Either load intrinsics from `INTRINSICS_PATH` (if `PRE_CALIBRATED=True`) or calibrate from saved frames and save intrinsics

## How to execute
- If you already have `camera_intrinsics_phone.npz`:
  - Set `PRE_CALIBRATED=True`
  - Run Block 2 → it loads `K`, `dist`, `image_size`
- If you need to calibrate:
  - Set `PRE_CALIBRATED=False`
  - Run Block 2:
    - Part (A): press SPACE to save multiple frames (15–30 recommended), ESC to quit capture
    - Part (B): runs chessboard detection on saved `.jpg` frames and calibrates

## Output
- When calibrating: writes `camera_intrinsics_phone.npz` with:
  - `camera_matrix` (K), `dist_coeffs` (dist), `image_size`
  - metadata: `pattern_size`, `square_length_m`, `board_outer_mm`, `cam_base`, `rms`

## Detection details (what it tries)
- Preprocess: CLAHE (contrast boost)
- Primary detector: `cv2.findChessboardCornersSB` on normal and inverted images
- Fallback: classic `findChessboardCorners` + `cornerSubPix`

## Common failure modes
- “No images found”: you didn’t capture frames or `CALIB_DIR` path is wrong
- “No usable chessboard detections”:
  - improve lighting, reduce glare, include more tilt/rotation, fill frame more


In [12]:
# ----------------------------
# (A) Capture frames (optional)
# ----------------------------
# Captures full-res frames from the phone stream into CALIB_DIR.
# Preview is mirrored and downscaled only for usability; saved images are original frames.
if not PRE_CALIBRATED:
    CALIB_DIR.mkdir(parents=True, exist_ok=True)

    cap, chosen_url, first = open_phone_cap()
    h0, w0 = first.shape[:2]
    print("Opened stream:", chosen_url)
    print(f"Stream resolution: {w0}x{h0}")
    print("SPACE=save | ESC=quit")

    idx = 0
    while True:
        ok, frame = cap.read()
        if not ok or frame is None:
            print("Frame read failed; exiting.")
            break

        preview = cv2.flip(frame, 1)

        ph, pw = preview.shape[:2]
        scale = min(PREVIEW_MAX_W / pw, PREVIEW_MAX_H / ph, 1.0)
        if scale < 1.0:
            preview = cv2.resize(
                preview,
                (int(pw * scale), int(ph * scale)),
                interpolation=cv2.INTER_AREA
            )

        cv2.imshow("PHONE: SPACE=save | ESC=quit", preview)
        key = cv2.waitKey(1) & 0xFF

        if key == 27:  # ESC
            print("ESC pressed; exiting.")
            break

        if key == 32:  # SPACE
            fname = CALIB_DIR / f"frame_{idx:04d}.jpg"
            ok_write = cv2.imwrite(str(fname), frame, [int(cv2.IMWRITE_JPEG_QUALITY), 95])
            print(f"Saved {fname}" if ok_write else f"Failed to save {fname}")
            idx += 1

    cap.release()
    cv2.destroyAllWindows()
else:
    print("Skipping capture (PRE_CALIBRATED=True)")


# ----------------------------
# (B) Calibrate or load intrinsics
# ----------------------------
# If PRE_CALIBRATED:
#   - Load K, dist, image_size from INTRINSICS_PATH
# Else:
#   - Detect chessboard corners on saved images
#   - Run cv2.calibrateCamera
#   - Save to INTRINSICS_PATH
if PRE_CALIBRATED:
    z = np.load(INTRINSICS_PATH, allow_pickle=True)
    K = z["camera_matrix"]
    dist = z["dist_coeffs"]
    image_size = tuple(z["image_size"])
    print("Loaded intrinsics from", INTRINSICS_PATH)
    print("image_size:", image_size)
else:
    objp = mk_object_points(PATTERN_SIZE, SQUARE_M)

    objpoints = []  # per-image 3D corner locations in board coordinates
    imgpoints = []  # per-image 2D detected corner locations in image coordinates
    image_size = None

    SB_FLAGS = (cv2.CALIB_CB_EXHAUSTIVE | cv2.CALIB_CB_ACCURACY)
    CLASSIC_FLAGS = (cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE)
    term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

    # CLAHE boosts local contrast, often improves chessboard detection on phone streams.
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))

    img_files = sorted(glob.glob(str(CALIB_DIR / "*.jpg")))
    if not img_files:
        raise RuntimeError(f"No images found in {CALIB_DIR}. Capture frames first (PRE_CALIBRATED=False).")

    for fname in img_files:
        img = cv2.imread(fname)
        if img is None:
            continue

        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Calibration assumes consistent image size for all samples.
        if image_size is None:
            image_size = (gray.shape[1], gray.shape[0])

        # Preprocess for detection.
        g = clahe.apply(gray)

        # Preferred: findChessboardCornersSB (robust) on normal and inverted.
        ok, corners = cv2.findChessboardCornersSB(g, PATTERN_SIZE, SB_FLAGS)
        if not ok:
            ok, corners = cv2.findChessboardCornersSB(255 - g, PATTERN_SIZE, SB_FLAGS)

        # Fallback: classic detector + subpixel refinement (normal only, as in your code).
        if not ok:
            ok, corners = cv2.findChessboardCorners(g, PATTERN_SIZE, CLASSIC_FLAGS)
            if ok:
                corners = cv2.cornerSubPix(g, corners, (11, 11), (-1, -1), term)

        if not ok:
            continue

        objpoints.append(objp.copy())
        imgpoints.append(corners)

    n = len(imgpoints)
    print("chessboard detections:", n, "image_size:", image_size)
    if n == 0:
        raise RuntimeError(
            "No usable chessboard detections. Improve lighting/contrast, reduce glare, "
            "fill frame more, add varied angles."
        )

    rms, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, image_size, None, None)

    np.savez(
        INTRINSICS_PATH,
        rms=float(rms),
        camera_matrix=K,
        dist_coeffs=dist,
        image_size=np.array(image_size, dtype=int),
        pattern_size=np.array(PATTERN_SIZE, dtype=int),
        square_length_m=float(SQUARE_M),
        board_outer_mm=float(BOARD_OUTER_MM),
        cam_base=str(CAM_BASE),
    )

    print("Saved intrinsics to", INTRINSICS_PATH)
    print("RMS reprojection error:", float(rms))


Skipping capture (PRE_CALIBRATED=True)
Loaded intrinsics from camera_intrinsics_phone.npz
image_size: (np.int64(1920), np.int64(1080))


# Block 3 — Undistortion sanity check (2×2 mosaic)

## Purpose
Visual diagnostic to verify intrinsics produce sensible undistortion:
- RAW frame
- UNDISTORTED frame (using `cv2.getOptimalNewCameraMatrix` + `cv2.undistort`)
- ABS DIFF (pixel difference magnitude)
- HOTSPOTS overlay (regions where undistortion changes pixels a lot)

## Preconditions
- `SHOW_COMPARISON=True`
- `K` and `dist` must already be defined (i.e., Block 2 has been run successfully)

## How to execute
- Choose source:
  - `SOURCE="phone_url"` uses `open_phone_cap()`
  - `SOURCE="device"` uses local webcam
- ESC to quit

## Interpreting output
- Some diff/hotspots near edges is normal (lens distortion strongest there)
- If hotspots dominate the whole image, intrinsics likely don’t match the actual camera source
  - e.g., calibrated on phone but viewing laptop webcam, or vice versa


In [13]:
if SHOW_COMPARISON:
    # Assumes: K, dist already exist (and match the camera you open here)

    # ----------------------------
    # Choose ONE source
    # ----------------------------
    SOURCE = "phone_url"      # "device" or "phone_url"

    # local webcam (used if SOURCE="device")
    CAM_INDEX = 0
    CAPTURE_W = 3840
    CAPTURE_H = 2160

    # phone base page (used if SOURCE="phone_url")
    CAM_BASE = CAM_BASE  # uses top-cell CAM_BASE by default

    # ----------------------------
    # Display layout
    # ----------------------------
    TILE_W = 640
    TILE_H = 360

    DIFF_THRESH = 25
    BLUR_K = 5
    ALPHA_NEWK = 0.0  # 0=crop to valid pixels (reduces black borders), 1=keep all pixels

    def open_device_cap():
        """
        Opens local webcam in MJPG mode at requested resolution.
        Returns: (cap, src_string, first_frame)
        """
        cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW)
        cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAPTURE_W)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAPTURE_H)
        if not cap.isOpened():
            cap.release()
            raise RuntimeError("Failed to open camera device")
        ok, frame = cap.read()
        if not ok or frame is None:
            cap.release()
            raise RuntimeError("Opened device camera but failed to read a frame")
        return cap, f"device:{CAM_INDEX}", frame

    def resize_tile(img):
        """Fixed-size tiles for a 2x2 mosaic."""
        return cv2.resize(img, (TILE_W, TILE_H), interpolation=cv2.INTER_AREA)

    def label(img, text):
        """Readable tile labels (white text with black outline)."""
        out = img.copy()
        cv2.putText(out, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0), 4, cv2.LINE_AA)
        cv2.putText(out, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 255), 2, cv2.LINE_AA)
        return out

    # ----------------------------
    # Open selected source
    # ----------------------------
    if SOURCE == "phone_url":
        cap, chosen_url, first = open_phone_cap()
        src = f"phone:{chosen_url}"
    elif SOURCE == "device":
        cap, src, first = open_device_cap()
    else:
        raise ValueError('SOURCE must be "phone_url" or "device"')

    print("ESC=quit | source:", src)

    # Cache newK by frame size.
    newK = None
    last_size = None

    while True:
        ok, frame = cap.read()
        if not ok or frame is None:
            break

        # ----------------------------
        # Undistort with an optimal new camera matrix (newK)
        # ----------------------------
        # getOptimalNewCameraMatrix chooses a new projection matrix that trades off:
        #   - cropping away invalid pixels (alpha=0)
        #   - vs keeping all pixels (alpha=1) which may add black borders
        h, w = frame.shape[:2]
        if newK is None or last_size != (w, h):
            newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), ALPHA_NEWK)
            last_size = (w, h)

        und = cv2.undistort(frame, K, dist, None, newK)

        # ----------------------------
        # Diff view: where undistortion changes pixels the most
        # ----------------------------
        g_raw = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        g_und = cv2.cvtColor(und, cv2.COLOR_BGR2GRAY)

        diff = cv2.absdiff(g_raw, g_und)

        # Optional blur to reduce speckle in diff/hotspot mask.
        if BLUR_K and BLUR_K > 0:
            k = BLUR_K if (BLUR_K % 2 == 1) else (BLUR_K + 1)
            diff = cv2.GaussianBlur(diff, (k, k), 0)

        diff_color = cv2.applyColorMap(diff, cv2.COLORMAP_TURBO)

        # ----------------------------
        # Hotspot mask: threshold + morphology + overlay
        # ----------------------------
        mask = (diff >= DIFF_THRESH).astype(np.uint8) * 255
        k5 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k5)
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, k5)

        overlay = und.copy()
        m = (mask > 0)

        # Blend red into hotspot pixels (same math as your code).
        if m.any():
            over_f = overlay.astype(np.float32)
            red = np.array([0, 0, 255], dtype=np.float32)
            over_f[m] = 0.6 * over_f[m] + 0.4 * red
            overlay = over_f.astype(np.uint8)

        hotspot_pct = 100.0 * float(np.count_nonzero(mask)) / float(mask.size)
        overlay = label(overlay, f"hotspots: {hotspot_pct:.2f}%")

        # ----------------------------
        # 2x2 mosaic
        # ----------------------------
        raw_t  = label(resize_tile(frame),      "RAW")
        und_t  = label(resize_tile(und),        "UNDISTORTED")
        diff_t = label(resize_tile(diff_color), "ABS DIFF")
        over_t = label(resize_tile(overlay),    "HOTSPOTS")

        top = np.hstack([raw_t, und_t])
        bot = np.hstack([diff_t, over_t])
        mosaic = np.vstack([top, bot])

        cv2.imshow("Calibration Sanity Check", mosaic)

        if (cv2.waitKey(1) & 0xFF) == 27:
            break

    cap.release()
    cv2.destroyAllWindows()


# Block 4 — Live measurement on chessboard plane (click 4 points → mm)

## Purpose
Runs an interactive measurement tool:
- Opens camera (phone stream first, otherwise webcam)
- Undistorts frames using saved intrinsics (`K`, `dist`)
- Maintains a stable chessboard pose estimate using:
  - periodic full re-detection (anchors)
  - LK optical-flow tracking between detections (stability)
  - forward-backward + max-jump gating (prevents blowups)
  - EMA smoothing of corners (reduces jitter)
  - RANSAC homography (robust to a few bad points)
- You click 4 points; if the board pose is valid, points are mapped to board-plane meters and width/height reported in mm

## Controls
- Left-click: add point / drag nearest existing point
- Right-click: toggle zoom window (use zoom to place points precisely)
- C: clear points
- P: print last measurement
- ESC: quit

## Preconditions
- `INTRINSICS_PATH` must exist and correspond to the camera you’re using
- Chessboard must be visible enough for detection (`PATTERN_SIZE`, `SQUARE_M` must match the real board)

## Output behavior
- Overlay shows:
  - board: OK / NOT FOUND
  - points: N/4
  - last measured W/H in mm (once 4 points are set and board pose is valid)
- On exit:
  - Raises if no measurement captured (no valid board + 4 points)
  - Prints final measured width/height (mm)

## Common failure modes
- board stays NOT FOUND:
  - improve contrast, reduce glare, fill more frame, hold steadier
  - consider lowering `DETECT_EVERY_N` (more frequent re-detection) or increasing `DETECT_SCALE` (less downscale)
- occasional pose “snaps”:
  - tighten gating (`FB_ERR_MAX_PX`, `MAX_JUMP_PX`) or increase `TRACK_MIN_FRAC`
  - increase smoothing slightly (`CORNER_EMA_ALPHA`)


In [ ]:
# ----------------------------
# Purpose
# ----------------------------
# Live measurement:
#   - Open camera (phone stream if available, else local webcam)
#   - Undistort frames using saved intrinsics (K, dist)
#   - Keep a stable chessboard pose estimate:
#       * Periodic re-detection of all corners (anchors the solution)
#       * Per-frame LK optical-flow tracking in between detections (reduces jitter)
#       * Robust gating (prevents “collapse to center” / “shoot off” failures)
#       * EMA smoothing of corner positions before computing homography (reduces jitter)
#       * RANSAC homography fit (a few bad points won’t trash H)
#   - You click 4 points; if board pose is valid, those 4 points are mapped to the board
#     plane (meters) and width/height are reported.

# ----------------------------
# Runtime state
# ----------------------------
zoom_on = False
zoom_center = None
zoom_win = "zoom"

pts = []            # clicked points in FULL-RES undistorted image coords (float x,y)
drag_i = None
cycle_i = 0
last_dims_m = None

newK = None
last_size = None

_last_zoom_origin = None
_frame_i = 0

# Board / homography state (FULL-RES undistorted image coords)
_have_H = False
_Hinv = None

# Corner state (N = PATTERN_SIZE[0] * PATTERN_SIZE[1])
_corners_full = None         # latest accepted corners (N,1,2) float32
_corners_smooth = None       # EMA-smoothed corners (N,1,2) float32
_prev_gray = None            # previous undistorted grayscale for LK tracking

# ----------------------------
# Load intrinsics
# ----------------------------
z = np.load(INTRINSICS_PATH, allow_pickle=True)
K = z["camera_matrix"]
dist = z["dist_coeffs"]

# ----------------------------
# Open camera (AUTO: phone stream else local webcam)
# ----------------------------
def _open_device():
    cap = cv2.VideoCapture(CAM_INDEX, cv2.CAP_DSHOW)
    cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, CAPTURE_W)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, CAPTURE_H)
    if not cap.isOpened():
        cap.release()
        return None, None
    ok, frame = cap.read()
    if not ok or frame is None or frame.size == 0:
        cap.release()
        return None, None
    return cap, frame

def open_cam_auto():
    # Try phone stream first (open_phone_cap() comes from your shared top cell)
    try:
        cap, url, frame = open_phone_cap()
        return cap, f"phone:{url}", frame
    except Exception:
        pass

    # Fallback to local device camera
    cap2, frame2 = _open_device()
    if cap2 is not None:
        return cap2, f"device:{CAM_INDEX}", frame2

    raise RuntimeError("Couldn't open phone stream OR local webcam.")

cap, src, first = open_cam_auto()
print("ESC=quit | C=clear | P=print | RClick toggle zoom | source:", src)

# ----------------------------
# Geometry / drawing helpers
# ----------------------------
def compute_display_scale(h, w):
    # Display scaling only; all computations remain in full-res coordinates.
    return min(DISPLAY_MAX_W / w, DISPLAY_MAX_H / h, 1.0)

def to_full_res(x, y, scale):
    # Mouse coords (display) -> full-res coords (undistorted frame space)
    return int(x / scale), int(y / scale)

def to_disp(x_full, y_full, scale):
    # Full-res coords -> display coords
    return int(x_full * scale), int(y_full * scale)

def nearest_point_index_full(x, y, pts_list, r=POINT_HIT_R_FULL):
    # Find nearest clicked point within hit radius (full-res coordinates)
    if not pts_list:
        return None
    p = np.asarray(pts_list, dtype=np.float32)
    d2 = (p[:, 0] - x) ** 2 + (p[:, 1] - y) ** 2
    i = int(np.argmin(d2))
    return i if d2[i] <= r * r else None

def order_quad(pts4):
    # Stable ordering for drawing/measuring: sort by angle around centroid, then rotate so top-left is first
    pts4 = np.asarray(pts4, dtype=np.float32)
    c = pts4.mean(axis=0)
    ang = np.arctan2(pts4[:, 1] - c[1], pts4[:, 0] - c[0])
    pts4 = pts4[np.argsort(ang)]
    s = pts4.sum(axis=1)
    i0 = int(np.argmin(s))
    return np.roll(pts4, -i0, axis=0)

def quad_wh_m(world_xy4):
    # Compute width/height as avg of opposite edges in board-plane metric coords
    p = np.asarray(world_xy4, dtype=np.float64)
    d01 = np.linalg.norm(p[1] - p[0])
    d12 = np.linalg.norm(p[2] - p[1])
    d23 = np.linalg.norm(p[3] - p[2])
    d30 = np.linalg.norm(p[0] - p[3])
    w = 0.5 * (d01 + d23)
    h = 0.5 * (d12 + d30)
    return w, h

def zoom_crop_bounds(img_shape, center_xy, radius):
    h, w = img_shape[:2]
    cx, cy = center_xy
    x1 = max(0, cx - radius); x2 = min(w, cx + radius)
    y1 = max(0, cy - radius); y2 = min(h, cy + radius)
    return x1, y1, x2, y2

def make_zoom_view(img_bgr, center_xy, radius, factor):
    # Crop around zoom_center in full-res, then scale up by zoom_factor for a zoom window
    x1, y1, x2, y2 = zoom_crop_bounds(img_bgr.shape, center_xy, radius)
    crop = img_bgr[y1:y2, x1:x2].copy()
    if crop.size == 0:
        return None, None
    zoom = cv2.resize(crop, None, fx=factor, fy=factor, interpolation=cv2.INTER_NEAREST)
    zh, zw = zoom.shape[:2]
    cv2.line(zoom, (zw // 2, 0), (zw // 2, zh), (0, 255, 0), 1)
    cv2.line(zoom, (0, zh // 2), (zw, zh // 2), (0, 255, 0), 1)
    return zoom, (x1, y1)

def draw_marker(img, x, y, color=(0, 255, 0), ring_r=POINT_RING_R, ring_th=POINT_RING_TH, center_r=POINT_CENTER_R):
    # Hollow ring + center dot marker; size is in screen pixels (constant in main + zoom)
    x = int(round(x)); y = int(round(y))
    cv2.circle(img, (x, y), ring_r, color, ring_th, lineType=cv2.LINE_AA)
    cv2.circle(img, (x, y), center_r, color, -1, lineType=cv2.LINE_AA)

def draw_text(img, lines, x=18, y=40, dy=34):
    # White text with black outline for readability
    for i, s in enumerate(lines):
        yy = y + i * dy
        cv2.putText(img, s, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (0, 0, 0), 5)
        cv2.putText(img, s, (x, yy), cv2.FONT_HERSHEY_SIMPLEX, 0.85, (255, 255, 255), 2)

# ----------------------------
# Mouse callbacks
# ----------------------------
main_win = "Measure: LClick add/drag | RClick zoom toggle | C clear | P print | ESC quit"

def mouse_zoom(event, x, y, flags, param):
    # Zoom window mouse uses zoom-factor mapping back to full-res coords
    global pts, drag_i, cycle_i, zoom_on, zoom_center, _last_zoom_origin

    if not zoom_on or zoom_center is None or _last_zoom_origin is None:
        return

    ox, oy = _last_zoom_origin
    fx = float(ox + (x / zoom_factor))
    fy = float(oy + (y / zoom_factor))

    if event == cv2.EVENT_RBUTTONDOWN:
        zoom_on = False
        zoom_center = None
        try:
            cv2.destroyWindow(zoom_win)
        except:
            pass
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        i = nearest_point_index_full(fx, fy, pts, r=POINT_HIT_R_FULL)
        if i is not None:
            drag_i = i
            pts[drag_i] = (fx, fy)
            return

        if len(pts) == 4:
            drag_i = cycle_i
            cycle_i = (cycle_i + 1) % 4
            pts[drag_i] = (fx, fy)
            return

        if len(pts) < 4:
            pts.append((fx, fy))
            return

    elif event == cv2.EVENT_MOUSEMOVE:
        if drag_i is not None:
            pts[drag_i] = (fx, fy)

    elif event == cv2.EVENT_LBUTTONUP:
        drag_i = None

def mouse_main(event, x, y, flags, param):
    # Main window mouse: add/drag points; right click toggles zoom
    global pts, drag_i, cycle_i, zoom_on, zoom_center, disp_scale

    fx, fy = to_full_res(x, y, disp_scale)

    if event == cv2.EVENT_RBUTTONDOWN:
        if not zoom_on:
            zoom_center = (fx, fy)
            zoom_on = True
            cv2.namedWindow(zoom_win)
            cv2.setMouseCallback(zoom_win, mouse_zoom)
        else:
            zoom_on = False
            zoom_center = None
            try:
                cv2.destroyWindow(zoom_win)
            except:
                pass
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        i = nearest_point_index_full(fx, fy, pts, r=POINT_HIT_R_FULL)

        if i is not None:
            drag_i = i
            pts[drag_i] = (float(fx), float(fy))
            return

        if len(pts) == 4:
            drag_i = cycle_i
            cycle_i = (cycle_i + 1) % 4
            pts[drag_i] = (float(fx), float(fy))
            return

        if len(pts) < 4:
            pts.append((float(fx), float(fy)))
            return

    elif event == cv2.EVENT_MOUSEMOVE:
        if drag_i is not None:
            pts[drag_i] = (float(fx), float(fy))

    elif event == cv2.EVENT_LBUTTONUP:
        drag_i = None

cv2.namedWindow(main_win)
cv2.setMouseCallback(main_win, mouse_main)

# ----------------------------
# Chessboard detection setup
# ----------------------------
# world_xy: board-plane coordinates of each inner corner in meters (same indexing as OpenCV detection output)
nx, ny = PATTERN_SIZE
world_xy = (np.mgrid[0:nx, 0:ny].T.reshape(-1, 2).astype(np.float32) * float(SQUARE_M))

SB_FLAGS = (cv2.CALIB_CB_EXHAUSTIVE | cv2.CALIB_CB_ACCURACY)
CLASSIC_FLAGS = (cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_NORMALIZE_IMAGE)
term = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-3)

clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)) if CLAHE_ON else None

def detect_board_fast(gray_full):
    # Detect corners on a downscaled image for speed, then scale coordinates back up.
    if DETECT_SCALE != 1.0:
        g = cv2.resize(gray_full, (0, 0), fx=DETECT_SCALE, fy=DETECT_SCALE, interpolation=cv2.INTER_AREA)
    else:
        g = gray_full

    # CLAHE improves local contrast; helps on phone streams / glarey boards.
    g1 = clahe.apply(g) if clahe is not None else g

    # Prefer SB detector (more robust); try normal then inverted.
    if SB_FIRST:
        ok, corners = cv2.findChessboardCornersSB(g1, PATTERN_SIZE, SB_FLAGS)
        if ok:
            corners = corners.reshape(-1, 2) / DETECT_SCALE
            return corners.reshape(-1, 1, 2).astype(np.float32)

        ok, corners = cv2.findChessboardCornersSB(255 - g1, PATTERN_SIZE, SB_FLAGS)
        if ok:
            corners = corners.reshape(-1, 2) / DETECT_SCALE
            return corners.reshape(-1, 1, 2).astype(np.float32)

    # Fallback classic detector (less robust), with subpixel refinement.
    ok, corners = cv2.findChessboardCorners(g1, PATTERN_SIZE, CLASSIC_FLAGS)
    if ok:
        corners = cv2.cornerSubPix(g1, corners, (11, 11), (-1, -1), term)
        corners = corners.reshape(-1, 2) / DETECT_SCALE
        return corners.reshape(-1, 1, 2).astype(np.float32)

    ok, corners = cv2.findChessboardCorners(255 - g1, PATTERN_SIZE, CLASSIC_FLAGS)
    if ok:
        corners = cv2.cornerSubPix(255 - g1, corners, (11, 11), (-1, -1), term)
        corners = corners.reshape(-1, 2) / DETECT_SCALE
        return corners.reshape(-1, 1, 2).astype(np.float32)

    return None

# ----------------------------
# Corner tracking (LK) + gating + smoothing + robust homography
# ----------------------------
# This is the part that prevents:
#   - “collapse to middle then expand”
#   - “one frame shoots corners far away”
#   - jitter from small frame-to-frame corner noise
#
# Approach:
#   * If we already have corners, track them frame-to-frame with LK.
#   * Validate tracks with forward-backward consistency + max jump.
#   * If too many corners are bad, declare tracking failed and re-detect (if enabled).
#   * Smooth accepted corners with EMA.
#   * Fit homography with RANSAC and invert it (image -> world).

N_CORNERS = int(PATTERN_SIZE[0] * PATTERN_SIZE[1])
MIN_GOOD_COUNT = int(np.ceil(float(TRACK_MIN_FRAC) * N_CORNERS))

LK_WIN = (21, 21)
LK_MAX_LEVEL = 3
LK_CRIT = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)

FB_ERR_MAX_PX = 1.25   # forward-backward error threshold (pixels)
MAX_JUMP_PX = 30.0     # max per-corner displacement per frame (pixels)

def ema_corners(prev_smooth, new_corners, alpha):
    # EMA in coordinate space; preserves corner ordering and reduces jitter.
    if prev_smooth is None:
        return new_corners.copy()
    a = float(alpha)
    return ((1.0 - a) * prev_smooth + a * new_corners).astype(np.float32)

def track_corners_robust(prev_gray, gray, corners_prev):
    """
    Returns:
      corners_next (N,1,2) float32  OR  None if tracking is considered failed
      good_mask    (N,) bool
    """
    p0 = corners_prev.reshape(-1, 1, 2).astype(np.float32)

    # Forward track: prev -> curr
    p1, st1, _ = cv2.calcOpticalFlowPyrLK(
        prev_gray, gray, p0, None,
        winSize=LK_WIN, maxLevel=LK_MAX_LEVEL, criteria=LK_CRIT,
        flags=0, minEigThreshold=1e-4
    )
    if p1 is None or st1 is None:
        return None, None

    # Backward track: curr -> prev (for forward-backward check)
    p0b, st2, _ = cv2.calcOpticalFlowPyrLK(
        gray, prev_gray, p1, None,
        winSize=LK_WIN, maxLevel=LK_MAX_LEVEL, criteria=LK_CRIT,
        flags=0, minEigThreshold=1e-4
    )
    if p0b is None or st2 is None:
        return None, None

    st1 = st1.reshape(-1).astype(bool)
    st2 = st2.reshape(-1).astype(bool)

    p0_xy = p0.reshape(-1, 2)
    p1_xy = p1.reshape(-1, 2)
    p0b_xy = p0b.reshape(-1, 2)

    fb_err = np.linalg.norm(p0_xy - p0b_xy, axis=1)       # how consistent is the track?
    disp = np.linalg.norm(p1_xy - p0_xy, axis=1)          # how far did it move this frame?

    good = st1 & st2 & (fb_err <= float(FB_ERR_MAX_PX)) & (disp <= float(MAX_JUMP_PX))
    good_n = int(good.sum())

    # Not enough reliable points -> treat as failed tracking
    if good_n < MIN_GOOD_COUNT:
        return None, good

    # Preserve ordering: any “bad” corner stays at previous location instead of poisoning H
    p1_fix = p1.copy()
    if (~good).any():
        p1_fix.reshape(-1, 2)[~good] = p0_xy[~good]

    return p1_fix.astype(np.float32), good

def update_hinv_from_corners(corners_full):
    """
    Computes H(world->image) with RANSAC, then stores Hinv(image->world).
    RANSAC prevents a small number of remaining bad points from collapsing H.
    """
    global _have_H, _Hinv

    if corners_full is None:
        _have_H = False
        _Hinv = None
        return

    img_pts = corners_full.reshape(-1, 2).astype(np.float32)

    H_world_to_img, inliers = cv2.findHomography(
        world_xy, img_pts, method=cv2.RANSAC, ransacReprojThreshold=2.0
    )
    if H_world_to_img is None or inliers is None:
        _have_H = False
        _Hinv = None
        return

    # Require enough inliers to trust the pose
    if int(inliers.sum()) < MIN_GOOD_COUNT:
        _have_H = False
        _Hinv = None
        return

    try:
        _Hinv = np.linalg.inv(H_world_to_img)
    except np.linalg.LinAlgError:
        _have_H = False
        _Hinv = None
        return

    _have_H = True

# ----------------------------
# Main loop
# ----------------------------
print("ESC=quit | C=clear | P=print | RClick toggle zoom")

while True:
    ok, frame = cap.read()
    if not ok or frame is None:
        break

    # (1) Undistort
    h, w = frame.shape[:2]
    if newK is None or last_size != (w, h):
        # newK is a per-resolution projection that makes undistortion stable (cached)
        newK, _ = cv2.getOptimalNewCameraMatrix(K, dist, (w, h), 1)
        last_size = (w, h)

    und = cv2.undistort(frame, K, dist, None, newK)
    gray = cv2.cvtColor(und, cv2.COLOR_BGR2GRAY)

    # (2) Update board pose:
    #     - re-detect periodically (anchors)
    #     - otherwise track from previous frame (stability)
    need_detect = (_corners_full is None) or ((_frame_i % DETECT_EVERY_N) == 0)

    if not need_detect and _prev_gray is not None and _corners_full is not None:
        tracked_corners, good_mask = track_corners_robust(_prev_gray, gray, _corners_full)

        if tracked_corners is None:
            # Tracking failed hard (too many bad corners)
            if REDETECT_ON_TRACK_FAIL:
                need_detect = True
            else:
                _have_H = False
                _Hinv = None
        else:
            # Tracking succeeded -> smooth -> update homography
            _corners_full = tracked_corners
            _corners_smooth = ema_corners(_corners_smooth, _corners_full, CORNER_EMA_ALPHA)
            update_hinv_from_corners(_corners_smooth)

    if need_detect:
        detected = detect_board_fast(gray)
        if detected is not None:
            _corners_full = detected
            _corners_smooth = ema_corners(_corners_smooth, _corners_full, CORNER_EMA_ALPHA)
            update_hinv_from_corners(_corners_smooth)
        else:
            _have_H = False
            _Hinv = None

    # Store current gray for next frame’s tracking
    _prev_gray = gray

    # (3) Display image (scaled), but keep all point math in full-res coords
    disp_scale = compute_display_scale(und.shape[0], und.shape[1])
    disp = und if disp_scale >= 1.0 else cv2.resize(
        und,
        (int(und.shape[1] * disp_scale), int(und.shape[0] * disp_scale)),
        interpolation=cv2.INTER_AREA
    )

    # Draw detected corners (use smoothed corners to avoid visible jitter)
    corners_for_draw = _corners_smooth if _corners_smooth is not None else _corners_full
    if corners_for_draw is not None:
        corners_disp = corners_for_draw.copy()
        corners_disp[:, 0, 0] *= disp_scale
        corners_disp[:, 0, 1] *= disp_scale
        cv2.drawChessboardCorners(disp, PATTERN_SIZE, corners_disp, True)

    # (4) Draw clicked points + quad; compute metric dims if homography is valid
    if len(pts) > 0:
        for i, (x, y) in enumerate(pts):
            dx, dy = to_disp(x, y, disp_scale)
            draw_marker(disp, dx, dy)
            cv2.putText(
                disp, str(i + 1),
                (dx + POINT_RING_R + 4, dy - POINT_RING_R - 2),
                cv2.FONT_HERSHEY_SIMPLEX, LABEL_SCALE, (0, 255, 0), 1, lineType=cv2.LINE_AA
            )

        if len(pts) == 4:
            quad = order_quad(pts)
            quad_disp = (quad * disp_scale).astype(np.float32)
            cv2.polylines(
                disp, [quad_disp.astype(np.int32).reshape(-1, 1, 2)],
                True, (0, 255, 0), 1, lineType=cv2.LINE_AA
            )

            if _have_H and _Hinv is not None:
                pts_img = quad.astype(np.float32).reshape(-1, 1, 2)
                # perspectiveTransform expects image points; Hinv maps image->world (board plane meters)
                pts_world = cv2.perspectiveTransform(pts_img, _Hinv).reshape(-1, 2)
                w_m, h_m = quad_wh_m(pts_world)
                last_dims_m = (float(w_m), float(h_m))

    # (5) Status overlay
    status = [
        f"board: {'OK' if _have_H else 'NOT FOUND'} | points: {len(pts)}/4 | zoom: {'ON' if zoom_on else 'OFF'}"
    ]
    if last_dims_m is not None:
        status.append(f"W: {last_dims_m[0] * 1000.0:.1f} mm   H: {last_dims_m[1] * 1000.0:.1f} mm")
    else:
        status.append("W: --   H: --")

    draw_text(disp, status)
    cv2.imshow(main_win, disp)

    # (6) Zoom window
    if zoom_on:
        if zoom_center is None:
            zoom_center = (w // 2, h // 2)

        zoom_img, origin = make_zoom_view(und, zoom_center, zoom_radius, zoom_factor)
        _last_zoom_origin = origin

        if zoom_img is not None and origin is not None:
            ox, oy = origin

            for i, (px, py) in enumerate(pts):
                zx = int((px - ox) * zoom_factor)
                zy = int((py - oy) * zoom_factor)
                if 0 <= zx < zoom_img.shape[1] and 0 <= zy < zoom_img.shape[0]:
                    draw_marker(zoom_img, zx, zy)
                    cv2.putText(
                        zoom_img, str(i + 1),
                        (zx + POINT_RING_R + 4, zy - POINT_RING_R - 2),
                        cv2.FONT_HERSHEY_SIMPLEX, LABEL_SCALE, (0, 255, 0), 1, lineType=cv2.LINE_AA
                    )

            cv2.imshow(zoom_win, zoom_img)

    # (7) Key handling
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break
    if key in (ord('c'), ord('C')):
        pts = []
        drag_i = None
        cycle_i = 0
        last_dims_m = None
    if key in (ord('p'), ord('P')):
        if last_dims_m is None:
            print("No measurement yet (need 4 points + board found).")
        else:
            print(f"W: {last_dims_m[0] * 1000.0:.1f} mm | H: {last_dims_m[1] * 1000.0:.1f} mm")

    _frame_i += 1

cap.release()
cv2.destroyAllWindows()

if last_dims_m is None:
    raise RuntimeError("No measurement captured (need 4 clicked points + board found).")

print("Measured dimensions:")
print(f"  Width : {last_dims_m[0] * 1000.0:.1f} mm")
print(f"  Height: {last_dims_m[1] * 1000.0:.1f} mm")


ESC=quit | C=clear | P=print | RClick toggle zoom | source: phone:http://192.168.1.70:8080/video
ESC=quit | C=clear | P=print | RClick toggle zoom
